In [1]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.5/117.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 22.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 24.4 

In [2]:
from google.colab import files
from google.colab import drive
import os
import random
import pathlib
import numpy as np
from numpy.random import seed
from PIL import Image

import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import image_classifier

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
%%capture
drive.mount('/content/gdrive', force_remount=True)
!tar --extract --verbose --file='/content/gdrive/MyDrive/Flowers_5K/training.tar'
!tar --extract --verbose --file='/content/gdrive/MyDrive/Flowers_5K/validation.tar'
!tar --extract --verbose --file='/content/gdrive/MyDrive/Flowers_5K/test.tar'

In [4]:
# Cartelle contenenti le immagini
train_folder = 'training/'
valid_folder = 'validation/'
test_folder = 'test/'

# Cartelle di destinazione per le immagini ridimensionate
output_train_folder = 'resized_train/'
output_valid_folder = 'resized_valid/'
output_test_folder = 'resized_test/'

# Dimensioni desiderate
target_size = (224, 224)

# Funzione per ridimensionare un'immagine e salvarla nella cartella di destinazione
def resize_image(image_path, output_path):
    image = Image.open(image_path)
    resized_image = image.resize(target_size)
    resized_image.save(output_path)

# Ridimensiona le immagini nella cartella di train
for class_folder in os.listdir(train_folder):
    class_path = os.path.join(train_folder, class_folder)
    output_class_folder = os.path.join(output_train_folder, class_folder)
    os.makedirs(output_class_folder, exist_ok=True)
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        output_image_path = os.path.join(output_class_folder, image_file)
        resize_image(image_path, output_image_path)

# Ridimensiona le immagini nella cartella di validazione
for class_folder in os.listdir(valid_folder):
    class_path = os.path.join(valid_folder, class_folder)
    output_class_folder = os.path.join(output_valid_folder, class_folder)
    os.makedirs(output_class_folder, exist_ok=True)
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        output_image_path = os.path.join(output_class_folder, image_file)
        resize_image(image_path, output_image_path)

# Ridimensiona le immagini nella cartella di test
for class_folder in os.listdir(test_folder):
    class_path = os.path.join(test_folder, class_folder)
    output_class_folder = os.path.join(output_test_folder, class_folder)
    os.makedirs(output_class_folder, exist_ok=True)
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        output_image_path = os.path.join(output_class_folder, image_file)
        resize_image(image_path, output_image_path)

In [5]:
train_data_dir = pathlib.Path('resized_train/')
val_data_dir = pathlib.Path('resized_valid/')
test_data_dir = pathlib.Path('resized_test/')

In [6]:
train_data = image_classifier.Dataset.from_folder(train_data_dir)
val_data = image_classifier.Dataset.from_folder(val_data_dir)
test_data = image_classifier.Dataset.from_folder(test_data_dir)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [7]:
spec = image_classifier.SupportedModels.EFFICIENTNET_LITE0
hparams = image_classifier.HParams(export_dir="exported_model", epochs=50)
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

In [8]:
model = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = val_data,
    options=options,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              3413024   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/50
1500/1500 [==============================] - 35s 17ms/step - loss: 0.5952 - accuracy: 0.6997 - val_loss: 0.3572 - val_accuracy: 0.9370
Epoch 2/50
1500/1500 [==============================] - 24s 16ms/step - loss: 0.3180 - accuracy: 0.9470 - val_loss: 0.2600 - val_accuracy: 0.9720
Epoch 3/5

In [9]:
model.export_model()

In [10]:
!ls exported_model
files.download('exported_model/model.tflite')

checkpoint  metadata.json  model.tflite  summaries


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
_, acc = model.evaluate(test_data)
print('Test accuracy: %f' % acc)

32/32 [==============================] - 3s 62ms/step - loss: 0.2330 - accuracy: 0.9830
Test accuracy: 0.983000
